# Nonlinear process model

Kalman filter processes like to use the following model:

$$
\def\M#1{{[\mathbf{#1}]}}
\def\MM#1#2{{[\mathbf{#1}{#2}]}}
\def\E{\operatorname{E}}
\def\cov{\operatorname{cov}}
\def\T{^\mathsf{T}}
$$
$$
\begin{eqnarray*}
\vec{x}_i&=&\M A\vec{x}_{i-1}&+\vec{w}_{i-1} \\
\vec{z}_i&=&\M H\vec{x}_i&+\vec{v}_i
\end{eqnarray*}$$

Nonlinear Kalman Filter processes tend to use a different but related form:

$$\begin{eqnarray*}
\vec{x}_i&=&\vec{f}(\vec{x}_{i-1})&+\vec{w}_{i-1} \\
\vec{z}_i&=&\vec{g}(\vec{x}_i)&+\vec{v}_i
\end{eqnarray*}$$

where

* $\vec{f}$ is the state transition function, which serves the same purpose as $\M A$. Given a previous state vector, the state transition function is used to transform it to the current state vector. So, this function takes a vector and returns a vector. This is more general than a pure matrix multiplication, and allows any kind of relation between this state and the next.
* $\vec{g}$ is the observation function, which serves the same purpose as $\M H$. Given a state, the observation function is used to transform it to the observations. Likewise, a vector function with vector parameter.

HOWEVER

Most physical laws are in the form of vector differential equations, so we have:

$$\frac{d\vec{x}}{dt}=\vec{F}(\vec{x},t)$$

where

* $\vec{F}$ is the physical law function, which is used to calculate the derivative of the state. This is then integrated to get a new function which describes the state in a continuous manner from the initial state and the laws. This function of state versus time is called the trajectory, since one of the basic uses of this is describing the actual trajectory of something, like a planet, moon, or robot. Remember not to confuse $\vec{f}$ with $\vec{F}$. We almost never use or even have access to $\vec{f}$.

Measurements are still taken at discrete instants. What we want is a way to shoehorn this continuous problem into a form suitable for the Kalman filter.

With sufficiently simple physics functions, you can crank out the integration as it shows in your textbooks and get the trajectory function directly. This is what [Newton invented calculus for](https://en.wikipedia.org/wiki/De_motu_corporum_in_gyrum) in the first place. However, for down-to-earth things like robots, the physics are usually too complicated to integrate directly. One of the things our filter will do for us is integrate numerically, so that we end up with (in principle) a list of times and the state vector for each time. This will be our trajectory function.

# Linearization

If you just want the recipe, skip a couple of sections.

Nonlinear problems are hard. In fact, they can't be done. So instead, we do a modified problem and hope that it is close enough. This is the part where everyone goes off on Taylor series and such. I will just talk somewhat informally.

First, most physical processes are smooth. No discontinuities and no corners. Just nice sweeping curves. If you look at any of these curves closely enough, the relative radius of curvature gets bigger and bigger, and any small section eventually appears linear if you zoom in on it enough.

In order to take advantage of this, you have to look at the right part of a curve. We need a reasonably accurate initial condition and a correct dynamics and kinematics model. Now we can use the derivative of the curve to find the general direction of motion. We use the initial condition estimate and model to create a reference trajectory. We end up not estimating the trajectory $\vec{x}(t)$ directly, but the deviation $\vec{X}(t)$ from the reference $\vec{x}_r(t)$ such that $\vec{x}_r(t)+\vec{X}(t)=\vec{x}(t)$.

So, we need a replacement for $\M A$ and a way to update $\hat{\vec{x}}_i$ and $\MM{P}{_i}$. The state vector is the easiest, so we'll talk about that. We know the physics model (If we don't, go find it!) so we can use it directly on our old estimate with numerical integration. I'll talk in detail about numerical integration in another post, but if you are a programmer, there is a good chance you have already written one. The simplest one is this simple:

* Find the rate of change of all the elements of the state vector at this point. That is, run $\vec{F}(\hat{\vec{x}}_{i-1})$, the physics function on the old estimate.
* Find the time step size. This is $\Delta t=t_i-t_{i-1}$.
* Multiply the rate of change vector by the time step size. This is how much the vector has changed over the step.
* Add the change to the old estimate to get the new estimate.

All numerical integrators are just improvements and refinements on this. If the time step is short enough, you can just do this. One simple refinement is to break the time step into smaller sub-steps and do the integration repeatedly.

Replacing $\M A$ is a bit harder. For one thing, we don't even need the transition matrix for the reference trajectory, but that for the *deviation* of the state from the projected reference state. This deviation decidedly does not follow the same dynamics as the state -- the closest analogy I can think of is the [Clohessy–Wiltshire equations](https://en.wikipedia.org/wiki/Clohessy%E2%80%93Wiltshire_equations), which describe the relative motion of two objects which are near each other in a circular orbit around a much larger third body. These equations will often describe the "orbit" of one body around the other, even though the actual gravity between them is negligible, or sometimes even literally zero if one of the bodies is actually a target point.

The point is, the dynamics of the state deviation are *spectacularly* not those of the state itself. Do not try to use the $\M A$ matrix below to calculate the state transition of the reference state -- it might not even start off in the correct direction, even for arbitrarily small time steps. I wish I could assign a different symbol to it, but in the context of this problem, $\M A$ makes sense since it fills the same job as in the linear filter, and there isn't a decorator which would make sense, since the name is already capitalized.

Note that while the dynamics of $\vec{X}$ and $\vec{x}$ are *different*, they are *related*. Since if you add $\vec{X}$ and $\vec{x}_r$, you get the actual propagated trajectory $\vec{x}$. Therefore, the uncertainty of $\hat{\vec{X}}$ is exactly the same as the uncertainty on $\hat{\vec{x}}$. You use the dynamics of the deviation $\vec{X}$ to propagate the covariance $\M P$, just as well as you could use the dynamics of $\vec{x}$. However, as we will see below, it is easier to propagate the deviation dynamics $\vec{X}$ in a way which is automatically derivable from the function $\vec{F}(\vec{x})$.

Another way to think about this is that $\M P$ describes a point cloud, a whole potentially-infinite population of deviations. When thought of this way, it is obvious that the dynamics of the deviation apply to the covariance.

Skipping all the justifications and derivations, you need a physics matrix $\M \Phi$. Finding it will require cranking out a bunch of derivatives by hand or computer, or writing a numerical differentiator. Then with $\M \Phi$ in hand, you can numerically integrate to get $\M A$. This is the "linearization" process talked about in textbooks.

Matrix $\M \Phi$ is a Jacobian matrix, a kind of vector derivative. Instead of taking the derivative with respect to time as you may be accustomed to, you take the derivative of each component of the physics function with respect to each of the other components. You end up with a matrix, where for each cell, you have the derivative of the component that is this cell's row number, with respect to the component that is this cell's column number. If you want to, you can think of it as fitting tangent hyperplane to the physics function, in the same way that an ordinary derivative fits a tangent line to a 1D function. This hyperplane is flat, so this is why we call it "linearized". We then project this hyperplane out as far as we need it, and use it to transform an identity matrix from one place on the trajectory to $\M A$ on another. The math is pretty straight forward, it's just hard to imagine, especially with more than about 1 element in your state vector.

Once you have $\M A$ in hand, you can easily find $\MM{P}{^-}$ using the same formula as before. The dynamics follows that of the deviation, but as we see above, while the mean of the deviation is far different from 

We end up playing a similar Jacobian trick with the nonlinear observation. The Jacobian matrix of the observation function $\vec{g}(\vec{x})$ is: $$\frac{d\vec{g}}{d\vec{x}}=\M H$$ No confusion with linear $\M H$ because we use it the same way ever after. Note however that this transforms the state deviation into the measurement deviation from that predicted by the reference state.

## Rectification
My first teacher for the Kalman filter talked about how the deviation could be maintained over multiple steps, so that we don't have to carry out the addition above, but if the deviation got too far away from the reference trajectory, the linear approximation would no longer be good enough and we would have to *rectify* the reference by actually doing the addition, resetting the deviation to zero, and I think resetting the deviation matrix to identity as well (If not rectifying, start the integration from where we left it last time). I never saw the point of not doing this addition each time (computational effort? Hard to justify given how much other work the filter does) and in a control loop, we need to do at least the addition each time. So, the code below will rectify after each measurement. My first teacher called this rectification at each step the *Extended Kalman filter*, as distinguished from the ordinary *Nonlinear Kalman filter* which maintained the deviation for more than one step. I haven't seen anyone else make this particular distinction.

# Extended Kalman Filter algorithm

To remember, the five equations of the linear Kalman filter are

$$\begin{eqnarray*}
\hat{\vec{x}}^- &=&\M A\hat{\vec{x}}_{i-1} & (1)\\
\MM{P}{^-}&=&\M A\MM{P}{_{i-1}}\M{A}\T+\M Q& (2)\\
\M{K}  &=&\MM{P}{^-}\M{H}\T(\M H\MM{P}{^-}\M H\T+\M R)^{-1}& (3)\\
\hat{\vec{x}}   &=&\hat{\vec{x}}^-+\M{K}(\vec{z}-\M H\hat{\vec{x}}^-)& (4)\\
\M{P}  &=&(\M 1-\M{K}\M H)\MM{P}{^-}& (5)
\end{eqnarray*}$$

Equations 1 and 2 are called the time update while equations 3 through 5 are called the measurement update.

Plugging in what we have above into this form, we get the following. Don't panic too much, it will all make sense with a concrete example.

## Time update

1a. Calculate the updated *reference* position $\hat{\vec{x}}^-$ by numerically solving: $$\frac{d\vec{x}}{dt}=\vec{F}(\vec{x}(t),t)$$
from $t=t_{i-1}$ to $t=t_i$ with initial condition: $$\vec{x}(t_{i-1})=\hat{\vec{x}}_{i-1}$$

1b. Calculate the *state deviation* transition matrix $\M A=\MM{A}{(t_i,t_{i-1})}$ by numerically solving: $$\frac{d\M{A}}{dt}=\MM{\Phi}{(\vec{x}(t),t)}\MM{A}{(t,t_{i-1})}$$ from $t=t_{i-1}$ to $t_i$ with initial condition: $$\MM{A}{(t_{i-1},t_{i-1})}=\M 1$$

Basically these integrations cover the time from one step to the next, generate our $\M A$ for us, and turn this from a continuous time problem back into a step problem like we had before. We do these integrations numerically, interleaved, and in lockstep, so that we can use the intermediate result $\vec{x}(t)$ from 1a where we need it in 1b. We will write  a function which smashes our vector $\vec{x}$ and matrix $\M A$ into a really tall augmented vector $\vec{x}_A$:

In [ ]:
import numpy as np

def smashxA(x,A):
    """
    Smash a vector and state transition matrix (or a set of vectors and matrices)
    into a vector or augmented set of vectors. If we get passed a set of m, 
    n-element vectors in the form of an nxm array, we expect also to get a set of
    same size m of nxn A matrices, in the form of an nxnxm array. The return result
    will be a 2D array of (n**2+n)xm size.
    """
    n=x.shape[0]
    if len(x.shape)>1:
        m=x.shape[1]
    else:
        m=1
    return np.vstack((x,A.reshape(-1,n**2).T))

We will also need a function which unsmashes $\vec{x}_A$ back into $\vec{x}$ and $\M A$. Unsmashing is a touch more difficult, because we have to figure out the size of the state and matrix from the size of the augmented state. Any state vector with $n$ elements will have a matching transition matrix of size $n \times n$, for a total of $a=n+n^2$ elements. We solve that for $n$ by plugging in the appropriate coefficients to the quadratic formula:

$$\begin{eqnarray*}
a&=&n+n^2 \\
0&=&n^2+n-a \\
n&=&\frac{-1\pm\sqrt{1+4a}}{2} \\
\end{eqnarray*}$$

So for instance, if $n=4$, we would have

$$\begin{eqnarray*}
a&=&n^2+n \\
 &=&20
\end{eqnarray*}$$

Running that back through the unsmasher gives:

$$\begin{eqnarray*}
n&=&\frac{-1\pm\sqrt{1+4a}}{2} \\
 &=&\frac{-1\pm9}{2} \\
 &=&\frac{9-1}{2} \\
 &=&\frac{8}{2} \\
 &=&4 \\
\end{eqnarray*}$$

So, we have to take the plus sign for the $\pm$.

In [ ]:
def unsmashxA(xA):
    """
    Reverse of smashxA()
    """
    #Figure out size
    a=xA.shape[0]
    n=int((np.sqrt(4*a+1)-1)/2+0.5) #Make sure it is rounded correctly. 
    x=xA[:n,...]
    A=xA[n:,...].T.reshape(-1,n,n)
    if A.shape[0]==1:
        A=A[0,:,:]
    return(x,A)

In [ ]:
#Test smashxA and unsmashxA with single vector components
x=np.array([[11],[21],[31],[41]])
A=np.array([[611,612,613,614],
            [621,622,623,624],
            [631,632,633,634],
            [641,642,643,644]])
xA=smashxA(x,A)
#print(xA)
print(unsmashxA(xA))

#Test them with multiple vector components
x=np.array([[11,12,13],[21,22,23],[31,32,33],[41,42,43]])
A=np.stack((np.array([[111,112,113,114],
                      [121,122,123,124],
                      [131,132,133,134],
                      [141,142,143,144]]),
            np.array([[211,212,213,214],
                      [221,222,223,224],
                      [231,232,233,234],
                      [241,242,243,244]]),
            np.array([[311,312,313,314],
                      [321,322,323,324],
                      [331,332,333,334],
                      [341,342,343,344]])))
xA=smashxA(x,A)
print(unsmashxA(xA))

Now we take as our derivative function for the augmented vector, a function which unsmashes the augmented vector, runs the real $\dot{\vec{x}}=\vec{F}(\vec{x})$ and $\dot{\M A}=\MM{\Phi}{(\vec{x})}\M A$ on them, then smashes them into $\dot{\vec{x}}_a$. We can use this with any first-order vector [ODE solver](ODESolver.ipynb). Often it doesn't make sense to do anything more complicated than a single-step Euler method, since the process noise and measurement noise will swamp the numerical error from the ODE solver. Any error in the ODE solver will be considered by the filter as process noise, and if we were really clever, we could incorporate this in $\M Q$.

Note: If you are using an ODE solver in a simulation where you are *generating* the truth, you want that to be as accurate as possible. Use a multi-step RK4, or even an analytical solution like in [UniversalVariables.ipynb](UniversalVariables.ipynb) for the two-body problem.

In [ ]:
def Fekf(xA,k,t=None):
    """
    Derivative function for EKF propagator of both state and state transition matrix
    :param xA: Augmented state with original state and state transition matrix smashed together
    :param k: Parameter vector. For this function, we need the following elements:
      0 - F function, function which calculates the derivative of the state vector
      1 - Phi function, function which calculates the Jacobian of F
      2: - Extra parameters for F and Phi
    :param t: Time in integration
    :return: Augmented state derivative, with the original state derivative and state transition
             matrix derivative smashed together
    """
    F=k[0]
    Phi=k[1]
    (x,A)=unsmashxA(xA)
    xd=F(x,k[2:],t)
    Ad=Phi(x,k[2:],t) @ A
    return smashxA(xd,Ad)

Continuing on with the filter, we do:

2. $\MM{P}{^-}=\M{A}\MM{P}{_{i-1}}\M{A}\T+\M Q$

Equation 2 is exactly like the old form. Combining steps 1a, 1b, and 2, we have code which does the time update:

In [ ]:
%%capture
%run ODESolver.ipynb

In [ ]:
def ekf_timeup(xh_im1, P_im1, t, dt, F, Phi,Fpar=None, Q=None, odesolve=rk4, nstep=10):
    """
    Do the Extended Kalman Filter time update. Given a previous state estimate, previous
    covariance, and everything else needed to do the physics, calculate the updated state
    estimate and covariance after taking a time step.
    :param xh_im1: Previous estimate of the state vector
    :param P_im1 : Previous estimate covariance
    :param dt    : time between last and current measurement
    :param F     : Physics function
    :param Phi   : Physics matrix function
    :param Fpar  : Extra parameters to pass to physics functions
    :param Q     : Process noise covariance matrix, square n x n where n is number of elements of state vector
    :return      : A tuple:
        xh_m - New estimated state vector
        P_m  - New estimate covariance matrix
    """
    # One numerical integrator for both 1A and 1B, so that 1B can use the
    # intermediate results of 1A.
    k=[F,Phi]
    if Fpar is not None:
        k=k+Fpar
    A0=np.eye(P_im1.shape[0]) #Will be integrated into state transition matrix
    (xh_m,A)=unsmashxA(odesolve(Fekf,dt,smashxA(xh_im1,A0),k=k,nstep=nstep,t0=t))

    # 2. Projected covariance estimate
    P_m = A @ P_im1 @ A.T
    if Q is not None:
        P_m+=Q
    return (xh_m,P_m)



## Measurement update

Equation 1c is where things start going different. We are estimating the deviation from the reference trajectory, but our update follows the reference trajectory perfectly, so our initial estimate of the deviation is zero. 

1c. $\hat{\vec{X}}^-=\vec{0}$

3a. $\M{H}=\frac{dg(\hat{\vec{x}}^-,t)}{d\hat{\vec{x}}^-}$ Linearized observation matrix is Jacobian of observation function

3b. $\M{K}=\MM{P}{^-}\M{H}\T(\M{H}\MM{P}{^-}\M{H}\T+\M R)^{-1}$ Kalman gain

4a. $\vec{Z}=\vec{z}-g(\hat{\vec{x}}^-,t)$ Measurement innovation *from the reference state*. Note that this includes some innovation from the fact that the state has deviated by an estimated amount, and some from the fact that our estimate of the deviation isn't perfect.

4b. $\hat{\vec{X}}=\hat{\vec{X}}^-+\M{K}(\vec{Z}-\M{H}\hat{\vec{X}}^-)$ Measurement update of state deviation

4c. $\hat{\vec{x}}=\hat{\vec{x}}^-+\hat{\vec{X}}_i$ Update of reference state vector

Note that if we rectify each time, we can forget about $\hat{\vec{X}}^-$ altogether by combining 4b and 4c:

$$\begin{eqnarray*}
\hat{\vec{X}}&=&\hat{\vec{X}}^-+\M{K}(\vec{Z}-\M{H}\hat{\vec{X}}^-) \\
             &=&\vec{0}+\M{K}(\vec{Z}-\M{H}\vec{0}) \\
             &=&\M{K}\vec{Z} \\
\hat{\vec{x}}&=&\hat{\vec{x}}^-+\hat{\vec{X}} \\
             &=&\hat{\vec{x}}^-+\M{K}\vec{Z} \\
\end{eqnarray*}$$

5 . $\MM{P}{_i}=(\M 1-\MM{K}{_i}\MM{H}{_i})\MM{P}{^-_i}$ Measurement update of state covariance

As noted before, we do the rectification each step. I don't know if it is just a matter of *not* reinitializing the deviation vector each time, or if we have to *not* reinitialize $\M A=\M 1$ each time.

In [ ]:
def ekf_measup(xh_m, P_m, z, g,H,gpar=None,R=None,t=None):
    """
    :param xh_m  : Estimate at current time but before taking into account current measurement
    :param P_m   : Estimate covariance under same conditions
    :param z     : current measurement vector. Must be a column vector, rather 
    :param g     : Measurement function
    :param H     : Measurement matrix function
    :param gpar  : Extra parameters to pass to measurement function
    :param R     : Measurement noise covariance matrix, square m x m where m is number of elements of measurement vector
    :return      : A tuple:
        xh - New estimated state vector, with measurement taken into account
        P  - New estimate covariance matrix under same conditions
    """
    def _inv(x):
        """
        Matrix inverse which also handles scalars properly
        :param x:
        :return:
        """
        try:
            return inv(x)
        except ValueError:
            return np.array([[1.0/x]])
    # 3a. Linearized observation matrix
    HH = H(xh_m, gpar, t)
    # 3b. Kalman gain
    K = P_m @ HH.T @ _inv(HH @ P_m @ HH.T + R)
    # 4a. Measurement deviation
    Z = z - g(xh_m, gpar, t)
    # 4c. Update of reference state vector
    xh = xh_m + K @ Z
    # 5. Measurement update of state covariance
    P = (np.eye(P_m.shape[0]) - K @ HH) @ P_m
    return (xh, P)


## Single step driver
The following code steps the filter forward one step by calling the two update routines above.

In [ ]:
def ekf_step(xh_im1, P_im1, z, dt, F, Fpar, g, gpar, Phi, H, Q, R, odesolve=rk4, nstep=10, measup=ekf_measup, t=None):
    """
    Call both the EKF time update and measurement update functions to process one measurement.
    :param xh_im1: Previous estimate of the state vector
    :param P_im1 : Previous estimate covariance
    :param z_i   : current measurement vector
    :param dt    : time between last and current measurement
    :param F     : Physics function
    :param Fpar  : Extra parameters to pass to physics function
    :param g     : Measurement function
    :param gpar  : Extra parameters to pass to measurement function
    :param Phi   : Physics matrix function
    :param H     : Measurement matrix function
    :param Q     : Process noise covariance matrix, square n x n where n is number of elements of state vector
    :param R     : Measurement noise covariance matrix, square m x m where m is number of elements of measurement vector
    :param odesolve: ODE solver to use, passed along to time update
    :param nstep : Number of substeps to use in the step, passed along to time update
    :param measup: Measurement update routine. Default uses full vector measurement update, including matrix inverse
    :return      : A tuple:
        xh_i - New estimated state vector
        P_i  - New estimate covariance matrix
    """
    if dt>0:
        (xh_m,P_m)=ekf_timeup(xh_im1=xh_im1, P_im1=P_im1, dt=dt, F=F, Phi=Phi, Fpar=Fpar, Q=Q, t=t)
    else:
        (xh_m,P_m)=(xh_im1,P_im1)
    return measup(xh_m=xh_m, P_m=P_m, z=z, g=g, gpar=gpar, H=H, R=R, t=t)


## Component-wise measurement update

Just like with the linear Kalman filter, we can do a component-wise measurement update with the extended Kalman filter. For this, we will use the Jacobians, etc to calculate $\M H$, but then consider each row $\MM{H}{_i}$ in turn.

Because we are dealing with a state deviation $\vec{X}$, we have the choice of either updating or not updating the state estimate after each measurement. I am going to choose to not do the rectification until after all measurements have been incorporated into the state deviation.

1. Time update as normal
2. Measurement update:
   - Calculate the full linearized $\M H$ from the Jacobians, at the time-updated state.
   - Calculate the full predicted observation deviation as the difference between the predicted and actual deviation vectors.
   - Zero out the estimated state deviation.
   - For each component $j$ of the vector, do the following:
     * Figure $\MM{H}{_j}$ by taking row $i$ of the full matrix $\M H$. The result will be an $1 \times n$ matrix, or an $n$-element row vector.
     * Figure $R_j$ (which will be a scalar now) as $R_j=\sigma_j^2$
     * Calculate $\Gamma_j$ (which will be scalar now)
     * Calculate $\MM{S}{_j}$ which will be a $n \times 1$ matrix, or a column vector
     * Calculate $\MM{K}{_j}=\MM{S}{_j}\frac{1}{\Gamma_j}$. Instead of doing a matrix inverse for $\M \Gamma$, we just do a scalar division.
     * Use $\MM{K}{_i}$ to do the measurement update for $\hat{\vec{X}}$ and $\M{P}$
     * Do not do a time update, because no time has elapsed between this component of the measurement vector and the next.
   - Do the rectification by adding the state deviation estimate back to the state estimate.
   


In [ ]:
def ekf_measup_comp(xh_m, P_m, z, g,H,gpar=None,R=None,t=None):
    """
    Do a component-wise measurement update. This has the same interface as ekf_measup
    """
    HH = H(xh_m, gpar, t)
    Z = z - g(xh_m, gpar, t)
    Xh = np.zeros(xh_m.shape)
    P=P_m
    m=z.size
    for j in range(m):
        H_j=HH[j,:]
        R_j=R[j,j]
        Z_j=Z[j]
        Gamma=H_j @ P @ H_j.T + R_j
        S=P @ H_j.T
        K =  S/Gamma
        Xh = Xh + K @ (Z_j - H_j @ Xh)
        P = (np.eye(P.shape[0]) - K @ H_j) @ P
    xh = xh_m + Xh
    return (xh, P)

# Driver
To actually run the filter, we set up the initial guess and noise estimates, then call `ekf_loop()` to actually run the Extended Kalman filter. It takes a lot of parameters, but none of them are complicated. The hardest part is just passing them in the correct order. If you have a whole bunch of measurements in a block that you want to do, this is the way to go. This function calls `ekf_step()` to actually run the equations we have derived. 

In this code, we have to be careful about the first step. In particular, what time is the estimate valid? The way we will do this seems weird, but is easiest to code. In order to line up the zero indexes of the input initial guesses, times, and measurements, we will treat ``t[0]`` as the time that the initial estimate ``xh_0`` and ``P_0`` are valid. Our output will have as many state estimates and covariances as there are times, so in particular ``xh[:,0]`` will be the estimate at ``t[0]``.

We have a choice to make here: We can either process the measurement 0 as occuring at the time of the initial estimate, meaning that the output vector zero will match the input measurement, or we can ignore the first measurement. It is actually easiest to code the latter, so that's what we will do. As a consequence, our output estimate zero will exactly match our input initial estimate.

In [ ]:
def ekf_loop(xh_0, P_0, z, t, F, Phi, Fpar, g, H, gpar, Q, R, RDiag=False):
    """
    Drive the Extended Kalman filter over a series of measurements.
    
    All parameters have default None values, even though None is not legal for most of them. This is to allow
    the use of named parameters.
    :param xh_0: Initial guess of the state vector
    :param P_0:  Initial guess of the estimate covariance
    :param z:    Stack of measurement vectors
    :param t:    array of times, same length as number of measurements
    :param F:    Physics function
    :param Phi:  Physics Jacobian function
    :param Fpar: Extra Parameters to physics function 
    :param g:    Measurement function
    :param H:    Measurement Jacobian function
    :param gpar: Extra parameters to measurement function
    :param Q: - Process noise covariance matrix, square n x n where n is number of elements of state vector
    :param R: - Measurement noise covariance matrix, square m x m where m is number of elements of measurement vector
    :param RDiag: - If True, treat the R matrix as diagonal and use component-wise measurement update. If None,
                    check the R matrix for diagonality and use the appropriate method automatically.
    :return: A tuple
      xh - list of estimated states
      P - list of estimate covariance matrices
    """
    if RDiag is None:
        #Check if a matrix is diagonal, from https://stackoverflow.com/a/43885215/
        RDiag=np.all(R == np.diag(np.diagonal(R)))
    n = t.size  # Number of observations
    xh = np.zeros((xh_0.size, n))  # History of all estimates (starts as initial estimate)
    xh[:, 0] = xh_0.ravel();
    P = np.zeros((n,xh_0.size, xh_0.size))  # History of all estimate covariances
    P[0, :, :] = P_0;
    for i in range(1, n):
        [xh_i, P_i] = ekf_step(xh_im1=xh[:,i-1].reshape(-1,1),P_im1=P[i-1,:,:],z=z[:,i].reshape(-1,1), dt=t[i]-t[i-1],
                               F=F, Phi=Phi, Fpar=Fpar, g=g, H=H,gpar=gpar,Q=Q, R=R, t=t[i],
                               measup=ekf_measup_comp if RDiag else ekf_measup)
        # Record the current estimates
        xh[:, i] = xh_i.ravel();
        P[i,:, :] = P_i;
    return (xh, P)


# Extended Kalman Filter for linear velocity model

Time for an example. We'll start really really simple, and just redo the linear velocity model.

First off, state vector. Basically the state vector is all the things that the system "remembers" about itself. Think about it as a simulation. Once we have a program, we run the simulation and freeze it at a certain point. What do we need to save so that when we restore, we have enough data to start the simulation back up? We'll use the cart on rails model as an example. We are working with a massive cart on smooth, level, perfectly frictionless rails. Therefore the cart is free to move in one dimension only. The very meaning of one dimension is that it takes only one number to describe the position, and we will call this number $p$. If we think of it as a function of time, we might say $p(t)$. We need $p$ in our state vector, but this does not yet completely describe the state. We also need to know the velocity. A cart at 4m down the track stopped is very different from a cart at 4m moving at 50m/s. As it turns out, this is all we need to know. This is all the cart "remembers" - everything else, such as our random gusts of wind, are instantaneous only. Any effect they have is only through changing the position and velocity. Therefore, $\vec{x}=\begin{bmatrix}p & v\end{bmatrix}\T$. Our state vector has two elements, $p$ (in units of meters from the origin, positive on one side and negative on the other) and $v$ (in units of meters per second, positive if moving one way and negative if the other).

Next, physics. We actually use Newton's first law here (normally problems start with the second law). The cart has no forces applied to it, so it continues in a straight line at constant speed. The derivative of the speed is zero (this is what constant speed means) and the derivative of the position is the speed, so we end up with this:

$$\begin{eqnarray*}
\frac{dv}{dt}&=&0\\
\frac{dp}{dt}&=&v\\
\frac{d\vec{x}}{dt}&=&\begin{bmatrix}x_v\\0\end{bmatrix}
\end{eqnarray*}$$

An aside on vector notation. If the components of a vector $\vec{?}$ are named, I will specify the element named $@$ by $?_@$ . If the components are ordered, and they always are, I will call the first component of $\vec{?}$, $?_0$. If the vector already has a subscript such as $\vec{?}_\#$, we will call a component of it $?_{\#,0}$. So, for our state vector $\vec{x}$ above, we can talk about $x_p$, $x_v$, and note that $x_0=x_p$. We have to be careful with the notation -- while $x_0$ is a (scalar) component of vector $\vec{x}$, we often notate the initial state as $\vec{x}_0$. If we wanted to reference the position component of this initial state, we would write it as $x_{0,0}=x_{0p}$ Vector components always have an order, but sometimes it is more clear to talk about the components by name if any, so I will freely use both notations. When I get to it, I will number matrix cells with two indexes, row first, then column, starting at 0. If it is necessary for clarity, I will use a comma between the components. If the matrix is itself indexed, I will include the matrix index first, then the row index, then the column. So, the upper-left cell of any matrix $\M ?$ is $?_{00}$, and the lower-left corner of a $99\times 99$ matrix $\MM{?}{_\#}$ is $?_{\#,98,0}$.


Believe it or not, we are done with the physics function.

$$\begin{eqnarray*}
\vec{F}(\vec{x})&=&\frac{d\vec{x}}{dt} \\
                &=&\begin{bmatrix}x_v \\ 0\end{bmatrix}
\end{eqnarray*}$$

Now for the observation function. The notation indicates a function with vector input and vector output, but in this case the observation vector is a single number, the position along the track. We accept that a single-element vector is the same as a scalar. So, we get

$$\vec{g}(\vec{x})=x_p$$

Again, that's it.

Now for Jacobians. We need a function that can calculate $\M \Phi$ given any $\vec{x}$, and to do this we need derivatives. We are going to do the derivatives once on paper, code them up in a function, and then that function will provide the actual numerical derivative whenever we need it.

Matrix $\M \Phi$ is a $2 \times 2$ matrix, since it needs to be compatible with $\M A$ (since we add $\M A$ and a fraction of $\M \Phi$ in the numerical integrator). There are only four elements, so we can go nice and slow. In the upper left cell of the matrix, we take the derivative of the zero component of the force function $F_0(\vec{x})=x_v$ with respect to the zero component of the state vector $x_p$ . Since this variable doesn't even occur in the formula, the derivative is zero and we have $\Phi_{00}=0$. Next, in the upper right cell, we have component zero of the force function $F_0(\vec{x})=x_v$ still, but this time we are going with respect to the 1 component of the state vector $x_1=x_v$. The derivative of any variable with respect to itself is 1, so we get $\Phi_{01}=1$. Bottom left cell uses $F_1(\vec{x})=0$ and $x_0=x_p$ so we have $\Phi_{10}=0$. To finish off, we have bottom right cell which uses $F_1(\vec{x})=0$ and $x_1=x_v$ so $\Phi_{11}=0$. As it happens this time, all the cells are constant, so our physics matrix function can just return a constant in response to any state vector 

$$\MM{\Phi}{(\vec{x})}=\begin{bmatrix}0 & 1 \\ 0 & 0\end{bmatrix}$$

for any $\vec{x}$. As we shall see in our next example, things are not always so easy.

The Jacobian for $\vec{g}(\vec{x})$ is even easier, but it is still a matrix. Any Jacobian of any vector parameter function with a vector return value has as many columns as the input vector has elements, and as many rows as the output vector has elements. So, in this case, 1 row by 2 columns. For $H_{00}$, we use $g_0(\vec{x})=x_p$ and $x_0=x_p$, so $H_{00}=dx_p/dx_p=1$. For $H_{01}$ we have $g_0(\vec{x})=x_p$ and $x_1=x_v$, so $H_{01}=dx_p/dx_v=0$

$$\M{H}=\begin{bmatrix}1 & 0\end{bmatrix}$$

To sum up, our problem is described as follows:


* State vector $\vec{x}=\begin{bmatrix}p \\ v\end{bmatrix}$
* Physics function $\vec{F}(\vec{x})=\begin{bmatrix}x_v \\ 0\end{bmatrix}$
* Observation function $\vec{g}(\vec{x})=x_p$
* Linearized physics matrix $\M \Phi=\begin{bmatrix}0 & 1 \\ 0 & 0\end{bmatrix}$
* Linearized observation matrix $\M H=\begin{bmatrix}1 & 0\end{bmatrix}$

Also, a word about a priori initial guesses. You will note that we always need an initial guess of our state vector at step 0 $\hat{x}_0$ as well as an initial guess of our uncertainty of our initial guess, $\MM{P}{_0}$. For some problems, it doesn't matter so much what the initial guess is, since the filter converges rapidly. Sometimes, though, it doesn't, and when this happens, the filter gets farther and farther from the correct answer instead of closer, and we say that the filter diverges. If this happens in a real vehicle, the thing may diverge from its intended course and converge with a wall, causing the parts of the vehicle to diverge from each other. Let's agree right now not to diverge. Your filter can handle the truth, as much truth as you can give it. Give it as good of an initial guess as possible.

In [ ]:
%matplotlib notebook
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
from scipy.linalg import inv

In [ ]:
def F(x,k,t):
    """
    Physics function F(<x>). Takes a state vector, returns the derivative of
    the state vector. Primary physics model definition
    """
    return np.array([[x[1,0]], [0]])

def Phi(x,k,t):
    """
    Physics matrix [Phi(<x>)]. Takes the state vector and returns the jacobian
    matrix. Secondary physics model definition, as it is derived from F() (on
    paper).
    """
    return np.array([[0, 1], [0, 0]])

def g(x,k,t):
    """
    Observation function g(<x>). Takes a state vector, returns an observation
    vector. Primary observation model definition
    """
    return x[0,0]

def H(x,k,t):
    """
    Observation matrix [H(<x>)]. Takes the state vector and returns the
    Jacobian matrix. Secondary observation model definition, as it is derived
    from g() (on paper).
    """
    return np.array([[1, 0]])

## Usage Example

The program is structured as follows. The driver function starts with a piece to simulate the cart. It will create one data point every 100ms for 10s, for a total of 100 points. Next we will add some noise to create the measurements. Then, we create the a priori initial guess.

Finally, we run the filter loop. Each iteration of the filter loop will process one observation and create one state estimate. The time stamp of this estimate is exactly the time of the measurement. The numerical integrators will be in line, instead of in a separate function to start with. Just to see how it's done, we will do a numerical integration time step of 1ms, so we need 100 loops to get from one measurement to the next.

All the other equations will be pretty much as written.

After we have run the filter loop, we will compare the estimates with the true noise-free state we started with.

In [ ]:
def nonlinear_velocity(Q=None,sig_a=1,tag=None):
    # Simulated actual trajectory. Stopped at the origin for first five seconds,
    # then constant acceleration such that position reaches 25m after 10
    # seconds.

    t = np.arange(0, 10, 0.1)  # Timestamp of each measurement
    pt = np.hstack((np.zeros(50), np.arange(0, 5, 0.1) ** 2))  # True position

    # We want random noise, but the *same* random noise each time we run
    nr.seed(3217)
    R = 0.5 ** 2  # Measurement noise covariance, 50cm on position
    pn = nr.randn(t.size) * np.sqrt(R)  # Position noise, standard deviation of 10cm.

    dt=0.1
    if Q is None:
        Q = np.array([[dt**4/4, dt**3/2], [dt**3/2, dt**2]])*sig_a**2  # Process noise covariance
    z = (pt + pn).reshape(1,-1)  # Observations

    xh_0 = np.array([0, 0])  # Initial state vector guess
    P_0 = np.array([[1, 0], [0, 1]])  # Initial state vector uncertainty, 1m in position and 1m/s in velocity
    (xh,P)=ekf_loop(xh_0=xh_0, P_0=P_0, z=z, t=t, F=F, Phi=Phi, Fpar=None, g=g, H=H, gpar=None, Q=Q, R=R, RDiag=False)

    #Return what is needed in order to make the plots
    return (t,pt,z[0,:],xh[0,:],P[:,0,0])

#Run with the *wrong* nonphysical Q
(tw,ptw,zw,xhw,Pw)=nonlinear_velocity(Q=np.array([[0.01,0],[0,0.01]]),tag="Qw")
plt.figure("Qwrong Velocity filter")
plt.plot(tw, ptw, 'r-', tw, zw, 'k*', tw, xhw, 'b-')
plt.plot(tw, xhw+np.sqrt(Pw)*3, 'b--')
plt.plot(tw, xhw-np.sqrt(Pw)*3, 'b--')
plt.figure("Qwrong Residuals")
plt.plot(tw, ptw - ptw, 'r-', tw, zw-ptw, 'k*', tw, xhw-ptw, 'b-')
plt.show()

Hmm, not so good. The estimate starts lagging behind the truth, and never catches up. This is more because the model doesn't fit the actual cart (which in this case has a rocket on it which lights at $t$=5sec). If the model could not be improved, then we will have to sacrifice smoothness and increase the process noise. In this form, we had an arbitrary $\M Q$, which wasn't physically derived. We will try a different (larger) $\M Q$ which is derived from the sigma-acceleration model used in BeginnerKalman.

In [ ]:
#Gradually increase process noise and plot each one
(t0,pt0,z0,xh0,P000)=nonlinear_velocity(sig_a=0)
(t1,pt1,z1,xh1,P001)=nonlinear_velocity(sig_a=1)
(t2,pt2,z2,xh2,P002)=nonlinear_velocity(sig_a=2)
plt.figure("Velocity filter")
plt.plot(tw, ptw, 'r-',label='Truth')
plt.plot(tw, zw, 'k*',label='Measurements')
plt.plot(tw, xhw, 'b-',label='Nonphysical Q')
plt.plot(t0,xh0,label='sig_a=0')
plt.plot(t1,xh1,label='sig_a=1')
plt.plot(t2,xh2,label='sig_a=2')
plt.legend()
plt.figure("Residuals")
plt.plot(tw, ptw - ptw, 'r-',label='Truth')
plt.plot(tw, zw-ptw, 'k*',label='Measurements')
plt.plot(tw, xhw-ptw, 'b-',label='Nonphysical Q')
plt.plot(t0,xh0-pt0,label='sig_a=0')
plt.plot(t1,xh1-pt1,label='sig_a=1')
plt.plot(t2,xh2-pt2,label='sig_a=2')
plt.ylim(-2,2)
plt.legend()
plt.show()


The filter with no process noise is not able to track the proper value at all, of course. All of the ones with process noise do follow the data with some lag, and the more process noise there is, the less the lag is. This is as expected, because the filter expects that the process noise has zero mean, while in reality the process noise is zero for the first 5 seconds, then constant and positive after then.

One would think that having a larger $\M Q$ would lead to less noise filtering but better average tracking of the true value, but that doesn't seem obvious in the data above. The green line with the largest $\M Q$ shows both better tracking of the truth, and not noticeably worse noise amplitude.

# Automating the Jacobian matrices
Again, this is overkill for this problem but allows us to make sure that things are doing what they are supposed to in the simple case, before we hand it the complex case.

We can use SymPy, the symbolic Python library, to do the derivatives for us automatically, and generate runnable functions. The code below takes a list of vector component symbols, a list of extra constant parameter symbols, and a list of symbolic expressions for each component of a vector function, and returns two Python functions:

1. A function which when called, returns the vector function at a particular state vector
2. A function which when called, returns the Jacobian of the vector function at a particular state vector

In [ ]:
import sympy
from IPython.display import display
def fjac(xsyms,paramsyms,Fsyms):
    """
    Create callable vector functions and Jacobian functions
    from symbolic equations
    :param xsyms    : Tuple or list of symbols for the names of each state vector component
    :param paramsyms: Tuple or list of symbols for the names of each variable used which is not
                      part of the state vector
    :param Fsyms    : Tuple or list of symbolic expressions for each component of the vector formula.
                      There must be a one-to-one correspondence between xsyms and Fsyms, and each
                      component must use only variables in xsyms and paramsyms.
    """
    xmat=sympy.Matrix(xsyms) #Note that these build column matrices out of 1D iterables
    Fmat=sympy.Matrix(Fsyms)
    dFdx=Fmat.jacobian(xmat)
    dFdx=sympy.simplify(dFdx)
    print("State vector: ")
    display(xmat)
    print("Parameters: ")
    display(paramsyms)
    print("Physics Function: ")
    display(Fmat)
    print("Physics Matrix: ")
    display(dFdx)
    #Generate a function from the symbolic equations. The function
    #will have a number of parameters equal to the number of symbols
    #in the vector component list, plus the number of symbols in the
    #parameter list
    if paramsyms is None:
        _F=sympy.lambdify(xsyms          ,Fmat)
    else:
        _F=sympy.lambdify(xsyms+paramsyms,Fmat)
    help(_F)
    #This isn't quite as clean as I had hoped. _jac has to be presented with scalars,
    #because the symbolic math will generate constant terms represented as scalars,
    #which will result in an array which has scalars in some elements and arrays in
    #others. Even a 1-element array will cause a problem, since it makes the whole
    #array no longer a rectangular lattice of numbers. _jac is the big problem, but
    #we call _F the same way.
    def F(x,params=None,t=None):
        """
        Physics function F(<x>). Takes a state vector, returns the derivative of
        the state vector with respect to time. Primary physics model definition
        :param x: state vector or group of state vectors. MUST BE a 2D numpy array,
                  no more, no less, even for a single column vector. Five is right out...
                  Must have exactly as many rows as xsyms. Rows will be passed to _F
                  in one-to-one correspondence to xsyms.
        :param params: Extra parameters to physics function, will be passed in one-to-one
                       correspondence with the paramsyms used to build _F
        :param t: Ignored, but required so this can be called by an ODE solver
        :return: State derivative vector, will be a 2D array the same shape as x.
        """
        result=None
        for i_col,col in enumerate(x.T):
            args=tuple(col)
            if params is not None:
                args=args+tuple(params)
            resultcol=_F(*args).ravel()
            if result is None:
                result=np.zeros((resultcol.size,x.shape[1]))
            result[:,i_col]=resultcol
        return result
    _jac=sympy.lambdify(xsyms+paramsyms,dFdx)
    help(_jac)
    def jac(x,params,t=None):
        """
        Physics matrix [\Phi(<x>)]. Takes a state vector, returns Jacobian of
        the state vector. Secondary physics model definition
        :param x: state vector or group of state vectors. MUST BE a 2D numpy array,
                  no more, no less, even for a single column vector. Five is right out...
                  Must have exactly as many rows as xsyms. Rows will be passed to _F
                  in one-to-one correspondence to xsyms.
        :param params: Extra parameters to physics function, will be passed in one-to-one
                       correspondence with the paramsyms used to build _jac
        :param t: Ignored, but required so this can be called by an ODE solver
        :return: Physics matrix group. Will be a 2D array if x is a single column
                 vector, or else a set with order [i_set,i_row,i_col]. (This index
                 order allows the @ operator to multiply two sets of matrices and
                 return a set of matrices)
        """
        result=None
        for i_col,col in enumerate(x.T):
            args=tuple(col)
            if params is not None:
                args=args+tuple(params)
            resultcol=_jac(*args)
            if result is None:
                result = np.zeros((x.shape[1], )+resultcol.shape)
            result[i_col,:,:]=resultcol
        if result.shape[0]==1:
            result=result[0,:,:]
        return result
    return (F,jac)



We can use this function both on the dynamics equations *and* the measurement equations:

In [ ]:
r,v=sympy.symbols('r v')
(F,Phi)=fjac([r,v],[],[v,0])
(g,H)=fjac([r,v],[],[r])

Note that these overwrite the hard-coded versions we used above. We can therefore run ``nonlinear_velocity`` and have it work, exactly as before:

In [ ]:
#Gradually increase process noise and plot each one
(t0,pt0,z0,xh0,P000)=nonlinear_velocity(sig_a=0)
(t1,pt1,z1,xh1,P001)=nonlinear_velocity(sig_a=1)
(t2,pt2,z2,xh2,P002)=nonlinear_velocity(sig_a=2)
plt.figure("Velocity filter2")
plt.plot(tw, ptw, 'r-',label='Truth')
plt.plot(tw, zw, 'k*',label='Measurements')
plt.plot(tw, xhw, 'b-',label='Nonphysical Q')
plt.plot(t0,xh0,label='sig_a=0')
plt.plot(t1,xh1,label='sig_a=1')
plt.plot(t2,xh2,label='sig_a=2')
plt.legend()
plt.figure("Residuals2")
plt.plot(tw, ptw - ptw, 'r-',label='Truth')
plt.plot(tw, zw-ptw, 'k*',label='Measurements')
plt.plot(tw, xhw-ptw, 'b-',label='Nonphysical Q')
plt.plot(t0,xh0-pt0,label='sig_a=0')
plt.plot(t1,xh1-pt1,label='sig_a=1')
plt.plot(t2,xh2-pt2,label='sig_a=2')
plt.ylim(-2,2)
plt.legend()
plt.show()
